In [81]:
#Importing the Libraries
import numpy as np
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt, numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import AgglomerativeClustering
from matplotlib.colors import ListedColormap
from sklearn import metrics
import warnings
import sys

PRE PROCESSING DATA

In [ ]:
df=pd.read_csv("ecommerce_customer_data - ecommerce_customer_data.csv.csv")
#dropping unecessary cols
df=df.drop(columns=["CustomerServiceInteractions","CustomerID","RepeatCustomer","PremiumMember","HasReturnedItems","RegistrationDate","SocialMediaEngagementRate","EmailEngagementRate","MobileAppUsage","TotalPurchases","CustomerLifetimeValue"])
#dropping null rows of cols
df = df.dropna(subset=["Age","IncomeLevel","Gender","SecondFavoriteCategory","FavoriteCategory","Country","City"])

#filling cols with mean of col
df["AverageSatisfactionScore"]=df["AverageSatisfactionScore"].fillna(df["AverageSatisfactionScore"].mean())
df["SocialMediaConversionRate"]=df["SocialMediaConversionRate"].fillna(df["SocialMediaConversionRate"].mean())
df["SearchEngineConversionRate"]=df["SearchEngineConversionRate"].fillna(df["SearchEngineConversionRate"].mean())
df["AverageOrderValue"]=df["AverageOrderValue"].fillna(df["AverageOrderValue"].mean())
df["EmailConversionRate"]=df["EmailConversionRate"].fillna(df["EmailConversionRate"].mean())

#converting text into numbers for faster calcutions by the algorithm
df['IncomeLevel'] = df['IncomeLevel'].replace({
    'High': 0.5,
    'Very High': 1,
    'Medium': 0,
    "Low": -1,
})
df["Gender"] = df["Gender"].replace({
    "Female":-1,
    "Male" : 1,
})
#removing neg ages
df = df[df['Age'] >= 0]

df['Gender'] = df['Gender'].apply(lambda x: 0 if isinstance(x, str) else x)

#Convert categorical variables to numerical using LabelEncoder
le = LabelEncoder()
df['Country'] = le.fit_transform(df['Country'])
df['City'] = le.fit_transform(df['City'])
df['FavoriteCategory'] = le.fit_transform(df['FavoriteCategory'])
df['SecondFavoriteCategory'] = le.fit_transform(df['SecondFavoriteCategory'])

#Scale numerical features
scaler = StandardScaler()
numerical_columns = ['Age', 'AverageOrderValue', 'AverageSatisfactionScore', 'EmailConversionRate', 
                     'SocialMediaConversionRate', 'SearchEngineConversionRate']

df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

print(df.head())








        Age  Gender  IncomeLevel  Country  City  AverageOrderValue  \
3  0.950813      -1          0.5        0     2          -0.112828   
4 -0.400539      -1          1.0        0     7          17.443615   
5  1.491353       1         -1.0        7     6          -0.110126   
7  1.491353       0          0.0        3     0          -0.093097   
8  0.545407       0          0.5        1     3          -0.008521   

   FavoriteCategory  SecondFavoriteCategory  AverageSatisfactionScore  \
3                 6                       2                  0.463291   
4                 2                       0                  0.435171   
5                 6                       7                 -1.571379   
7                 5                       7                  0.943098   
8                 3                       6                  0.004028   

   EmailConversionRate  SocialMediaConversionRate  SearchEngineConversionRate  
3            -1.047689                   0.061700           

C:\Users\moksh\AppData\Local\Temp\ipykernel_23716\1594687930.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['IncomeLevel'] = df['IncomeLevel'].replace({


DB SCAN